In [ ]:
# --- 1. Environment Setup ---
# This cell handles all installations and path configurations for Colab or local.
import os
from openstudio_toolkit.utils.notebook_setup import setup_environment

# The project root is assumed to be one level above the current working directory.
# We will construct an absolute path from this.
project_root_relative = "../.." 
project_root_abs = os.path.abspath(project_root_relative)
setup_environment(project_folder_path=project_root_abs)

# --- 2. Imports ---
import pandas as pd
from openstudio_toolkit.utils.osm_utils import load_osm_file_as_model, save_model_as_osm_file
from openstudio_toolkit.tasks.model_setup import rename_surfaces_based_on_space_names
from openstudio_toolkit.osm_objects.surfaces import get_all_surface_objects_as_dataframe

# --- 3. User Inputs ---
# This notebook is pre-configured to run with an example file.
example_file_name = "Cabana-60.osm"

# Construct the full, ABSOLUTE path to the example OSM file.
osm_file_path = os.path.abspath(os.path.join(project_root_abs, "examples", example_file_name))

# Define where the output file will be saved (also as an absolute path).
output_file_name = f"{os.path.splitext(example_file_name)[0]}_renamed.osm"
# os.getcwd() gives the current directory of the notebook file
output_osm_path = os.path.join(os.getcwd(), output_file_name)

print(f"INFO: Using example model: {osm_file_path}")
print(f"INFO: Output will be saved to: {output_osm_path}")

# --- 4. Load Model ---
# Check if the file exists before attempting to load
if not os.path.exists(osm_file_path):
    print(f"❌ ERROR: File not found at the specified path: {osm_file_path}")
    print("Please ensure the 'examples' folder and the .osm file exist in your project root.")
else:
    osm_model = load_osm_file_as_model(osm_file_path)

    # --- 5. Validate Model ---
    print("\n--- Validating Model for Rename Surfaces Task ---")
    report = rename_surfaces_based_on_space_names.validator(osm_model)
    for msg in report['messages']:
        print(msg)
    print("---------------------------------------------")

    # --- 6. Run Task and Save ---
    if report['status'] == 'READY':
        modified_model = rename_surfaces_based_on_space_names.run(osm_model)
        save_model_as_osm_file(modified_model, output_osm_path)
        print(f"\n✅ Model saved successfully to: {output_osm_path}")

        # --- 7. Verification Step ---
        print("\n--- Verifying Results ---")
        print("Loading the modified model to check the new surface names...")
        
        verified_model = load_osm_file_as_model(output_osm_path)
        final_surfaces_df = get_all_surface_objects_as_dataframe(verified_model)
        
        print("Displaying the first 5 surfaces from the modified model:")
        display(final_surfaces_df.head())

    else:
        print("\n❌ Task did not run due to validation errors.")